In [14]:
from langchain_core.prompts import ChatPromptTemplate

# 번역 기능을 위한 시스템 메시지 설정
'''
language를 stremalit에서 선택할 수 있게 만들어서 선택한 값으로 {language}자리에 입력할 수 있게 하면
깔끔하게 언어 설정이 될 것 같아요
(streamlit에서 선택은 영어로 표현해주시는 게 더 좋을 것 같습니다! ex. korean, english etc.)
'''

translate_template1 = ChatPromptTemplate.from_messages([
    ("system", "Translate the following sentence from {language} to korean:"),
    ("user", "{text}")
])

translate_template2 = ChatPromptTemplate.from_messages([
    ("system", "Translate the following sentence from Korean to {language}:"),
    ("user", "{text}")
])

translate_chain1 = translate_template1 | model  # 사용자 입력 -> 모델
translate_chain2 = translate_template2 | model  # 모델 -> 출력

language = input('choose your language')

In [18]:
# 챗봇 구동
query = ''
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")

    if query == 'stop':
        print("RAG 실행을 종료합니다.")
        break

    query_text = translate_chain1.invoke(
        {"language": language, "text": query})
    query = query_text.content if hasattr(query_text, "content") else query_text

    # 1. Retriever로 관련 법률 검색
    response_docs = rag_chain_debug["context"].invoke({"question": query})

    # 1-1. Retriever로 관련 상황 문서 검색
    response_docs1 = rag_chain_debug["context1"].invoke({"question": query})

    # 1-2. 관련 문서와 관련된 과실 비율 문서 검색
    response_docs2 = find_most_similar_doc(response_docs1[0].metadata['summary'].content)

    # 2. 문서를 프롬프트로 변환
    prompt_messages = contextual_prompt.format_messages(
        context=response_docs[0].page_content,
        context1=response_docs1[0].page_content,
        context2=response_docs2.page_content,
        question=query
    )

    # 3. LLM으로 응답 생성
    response = rag_chain_debug["llm"].invoke(prompt_messages)

    response = translate_chain2.invoke(
        {"language": language, "text": response})



    print("\n답변:")
    print(response.content)


답변:
I'm sorry to hear about the accident you experienced. The accident situation you mentioned is "an incident where a driver who exited the vehicle was struck by a following vehicle." In this case, since the following vehicle struck the driver who had exited, the fault of the following vehicle is recognized.

Based on the provided legal principles, it can be concluded that when a following vehicle strikes a driver who has exited, the driver of the following vehicle generally bears greater fault. Additionally, considering that in similar accident cases, the fault ratio of the claimant has been recognized as 100%, it is judged that the driver of the following vehicle should bear most of the fault.

Therefore, in your accident situation, the fault ratio of the following vehicle is assessed to be 100%, while the fault ratio of the exited driver is assessed to be 0%.

Compared to actual case law, this judgment is similar to cases where the fault of the following vehicle was recognized in 